# Cephノード構築：cephadm configファイル配置
ovs-cetlが参照する、ceph.confとadmin keyringファイルをすべてのノードに配置する<br>


# パラメータ定義

## パラメータファイル設定

Cephクラスタの定義ファイルを指定するため、<br>
パラメータ定義格納ディレクトリを表示する。

In [ ]:
!ls -p ~/notebooks/share/ | grep "/$"

In [ ]:
!ls -p ~/notebooks/share/configuration

In [ ]:
!ls -p ~/notebooks/share/configuration/ceph

In [ ]:
import sys, os.path, importlib, json, yaml

config_file_name = "ceph_admin_vm.yml"
config_file_path = os.path.expanduser("~/notebooks/share/configuration/ceph/{}".format(config_file_name))
with open(config_file_path) as f:
    admin_params = yaml.safe_load(f)
admin_params

In [ ]:
target_vm_name = 'xxx-ceph-admin1'
admin_patam = admin_params[target_vm_name]
admin_patam

In [ ]:
ceph_admin_ip = admin_patam['network']['provisioning']['ip']

In [ ]:
import sys, os.path, importlib, json, yaml

config_file_name = "xxx-openstack-parameter.yml"
config_file_path = os.path.expanduser("~/notebooks/share/configuration/{}".format(config_file_name))
with open(config_file_path) as f:
    params = yaml.safe_load(f)
params

In [ ]:
repo_server = params['repository']['addresses']['service_operation']
repo_server

In [ ]:
provisioning_vip = params['openstack']['provisioning_vip']
provisioning_vip

In [ ]:
network_segment_controller_vip = params['network_segment_controller_vip']
network_segment_controller_vip

## CephノードとストレージセグメントIPアドレス対応表の読み込み

In [ ]:
import os
import csv
import pandas as pd

instance_ip_csv_path =  os.environ['HOME'] + "/notebooks/share/configuration/ceph/ceph_instance_ip.csv"
instance_ip_list = []
instance_ip_map = {}

with open(instance_ip_csv_path, 'r') as path_csv:
    reader = csv.DictReader(path_csv)
    
    for record in reader:
        instance_ip_list.append(record)
        instance_ip_map[record['instance_name']] = record

pd.DataFrame(instance_ip_list)

## 作業用ディレクトリを作成する

In [ ]:
import os,tempfile
temp_dir = tempfile.mkdtemp()
print (temp_dir)

## AnsibleのInventoryファイルを作業用ディレクトリへ作成する

In [ ]:
import os

with open( os.path.join(temp_dir, "hosts"), 'w') as f:
    f.write('''[openstack_ctl]
{openstack_cmn}
'''.format(openstack_cmn = provisioning_vip))

hosts = temp_dir + "/hosts"
!cat $hosts

In [ ]:
%env ANSIBLE_INVENTORY={hosts}

疎通確認

In [ ]:
!ansible -m ping openstack_ctl

In [ ]:
!ansible -a 'hostname' openstack_ctl

## OpenStack用OpenRCファイル設定

`xxxxxxx`プロジェクト用のOpenRCファイルを事前にダウンロードしておくこと。  

In [ ]:
import os

user_creds = os.path.expanduser('~/.keys/xxxxxxx-openrc.sh')

assert os.path.exists(user_creds), '{} is not exist'.format(user_creds)

In [ ]:
%env USER_CREDS={user_creds}

In [ ]:
!ls -l {user_creds}

## openstackコマンド用ユーティリティ関数

In [ ]:
def build_cmdline(param):
    cmdline = ''
    
    for param_name, param_value in param.items():
        if isinstance(param_value, dict):
            for key, value in param_value.items():
                cmdline += ' --{} {}={}'.format(param_name, key, value)
        elif isinstance(param_value, list):
            for item in param_value:
                cmdline += ' --{} {}'.format(param_name, item)
        elif isinstance(param_value, bool):
            if param_value:
                cmdline += ' --{}'.format(param_name)
        else:
            cmdline += ' --{} {}'.format(param_name, param_value)

    return cmdline

In [ ]:
import json

def run_openstack(cmd):
    print('EXEC: {}'.format(cmd))
    out = !source {user_creds} && {cmd}
    print('OUTPUT: {}'.format('\n'.join(out)))
    return json.loads('\n'.join(out))

テスト

In [ ]:
run_openstack('openstack flavor list -f json' + build_cmdline({'all': True}))

## デプロイされたCephノード情報の取得

ストレージノードを特定するにあたり、ノード名称が　ceph-nodexxx （xxxの部分は、右寄せ0埋めの数字3桁）というルールに従っていることを前提としている。<br>
 例：ceph-node001<br>
 このノード名称ルールは、構築するCephクラスタ毎に変えるため、以下で定義する。

In [ ]:
# 初期化対象のインスタンス名称を設定する
raise Exception('この先手作業が必要です')

In [ ]:
import pandas as pd
import re

# 対象インスタンス名称マッチングルール
name_match_rule = r'ceph-xxx-node-[0-9]+'

## OpenStack Ironicからノード情報の取得

In [ ]:
cmd = "openstack baremetal node list -f json --long"
nodes = run_openstack(cmd)

In [ ]:
import pandas as pd
import re

for node in nodes:
    instance_uuid = node.get("Instance UUID", None)
    if instance_uuid is None:
        continue
        
    node["instance_name"] = node["Instance Info"]["display_name"]

pd.set_option('display.max_rows', None)
pd.DataFrame(nodes, columns=['Instance UUID', 'instance_name', 'UUID', 'Maintenance', 'Provisioning State', 'Power State', 'Name'])

In [ ]:
cmd = "openstack server list -f json --long"
nova_server_map = run_openstack(cmd)

In [ ]:
node_list = []
for server in nova_server_map:
    node_id = server['ID']
    if re.match(name_match_rule, server['Name']) is None:
        continue
    for node in nodes:
        if node['Instance UUID'] == node_id:
            node_info = {}
            node_list.append(node_info)
            node_info['name'] = server['Name']
            node_info['node_name'] = node['Name']
            instance_ip_info = instance_ip_map.get(server['Name'], None)
            if instance_ip_info is not None:
                node_info['instance_ip'] = instance_ip_info.get('ipv4', None)
            else:
                node_info['instance_ip'] = None
            address = server['Networks']
            prov_net = address['provisioning-net']
            for addr in prov_net:
                node_info['prov_ip'] = addr

pd.DataFrame(node_list)

## 登録対象Cephノード情報の取得

初期化対象のインスタンス名称を設定する

In [ ]:
# 初期化対象のインスタンス名称を設定する
raise Exception('この先手作業が必要です')

### 初期化対象ノードの指定

In [ ]:
# bootstrapノードを除いた、対象ノードを指定
target_name_list = ['ceph-xxx-node-002','ceph-xxx-node-003','ceph-xxx-node-004','ceph-xxx-node-005','ceph-xxx-node-006','ceph-xxx-node-007']

In [ ]:
bootstrap_node_name = 'ceph-xxx-node-001'

In [ ]:
target_node_list = []
bootstrap_node = None
for target_name in target_name_list:
    for elem in node_list:
        if elem['name'] == target_name:
            target_node_list.append(elem)
            break
for elem in node_list:
    if elem['name'] == bootstrap_node_name:
        bootstrap_node = elem
        break

assert (len(target_node_list) > 0), "指定されたターゲットノードが存在しません。"
assert (bootstrap_node is not None), "指定されたBootstrap nodeが存在しません。"

print(bootstrap_node)

In [ ]:
pd.DataFrame(target_node_list)

# Inventoryファイルを生成する。

In [ ]:
node_user='xxxxx'
cephadmin_user='cephadmin'
node_key='~/.ssh/id_ras'
proxy_user='xxxxx'
proxy_key='~/.ssh/id_ras'

In [ ]:
bootstrap_ip = bootstrap_node['prov_ip']

In [ ]:
import os

with open( os.path.join(temp_dir, "hosts"), 'w') as f:

    f.write('''[ceph_admin]
{ceph_admin_ip} ansible_user={proxy_user} ansible_ssh_private_key_file={proxy_key} ansible_python_interpreter=/usr/bin/python3
'''.format(ceph_admin_ip=ceph_admin_ip, proxy_user=proxy_user, proxy_key=proxy_key, bootstrap_ip=bootstrap_ip))

    f.write('''[bootstrap]
{node_ip} ansible_user={node_user} ansible_ssh_private_key_file={node_key} ansible_ssh_common_args='-o ControlMaster=auto -o StrictHostKeyChecking=no -o ControlPersist=30m -o ProxyCommand="ssh -W %h:%p -i {proxy_key} -q {proxy_user}@{ceph_admin_ip}"' ansible_python_interpreter=/usr/bin/python3
[ceph_nodes]
'''.format(ceph_admin_ip=ceph_admin_ip, node_ip=bootstrap_ip, node_key=node_key,
           node_user=node_user, proxy_user=proxy_user, proxy_key=proxy_key))
    
    for node in target_node_list:
        node_ip = node['prov_ip']
        f.write('''{node_ip} ansible_user={node_user} ansible_ssh_private_key_file={node_key} ansible_ssh_common_args='-o ControlMaster=auto -o StrictHostKeyChecking=no -o ControlPersist=30m -o ProxyCommand="ssh -W %h:%p -i {proxy_key} -q {proxy_user}@{ceph_admin_ip}"' ansible_python_interpreter=/usr/bin/python3
'''.format(ceph_admin_ip=ceph_admin_ip, node_ip=node_ip, node_key=node_key,
           node_user=node_user, proxy_user=proxy_user, proxy_key=proxy_key))

hosts = temp_dir + "/hosts"

!cat $hosts

In [ ]:
%env ANSIBLE_INVENTORY={hosts}

## 疎通確認
以下コマンドを実行し、OpeHubからCephAdmin経由で、Cephノードに疎通することを確認する。

In [ ]:
!ansible -b -m shell -a 'hostname' ceph_admin

In [ ]:
!ansible -b -m shell -a 'hostname' ceph_nodes

In [ ]:
for node in target_node_list:
    node_ip = node['prov_ip']
    !ansible -b -m shell -a 'hostname' $node_ip

In [ ]:
!ansible -b -m shell -a 'hostname' bootstrap

# ceph.confとadmin keyringファイルの取得と配置

## MON状態確認
MONが規定数生成されていることを確認

In [ ]:
!ansible -b -m shell -a "cephadm shell -- ceph orch ls 2> /dev/null" bootstrap

In [ ]:
!ansible -b -m shell -a "cephadm shell -- ceph -s 2> /dev/null" bootstrap

## ceph.confとadmin keyringファイルの取得

In [ ]:
!ansible -b -m shell -a 'cephadm shell -- ceph auth get client.admin > /tmp/ceph.client.admin.keyring' bootstrap

In [ ]:
!ansible -b -m shell -a 'cat /tmp/ceph.client.admin.keyring' bootstrap

In [ ]:
!ansible  -b -m shell -a 'cephadm shell -- ceph config generate-minimal-conf > /tmp/ceph.conf 2> /dev/null' bootstrap

In [ ]:
!ansible -b -m shell -a 'cat /tmp/ceph.conf' bootstrap

In [ ]:
!ansible -b -m fetch -a 'src=/tmp/ceph.client.admin.keyring dest={temp_dir}' bootstrap

In [ ]:
!ansible -b -m fetch -a 'src=/tmp/ceph.conf dest={temp_dir}' bootstrap

In [ ]:
!ls -la {temp_dir}/{bootstrap_ip}/tmp

In [ ]:
!cat {temp_dir}/{bootstrap_ip}/tmp/ceph.client.admin.keyring

In [ ]:
!cat {temp_dir}/{bootstrap_ip}/tmp/ceph.conf

## ceph.confとadmin keyringファイルの配置

In [ ]:
!ansible -CDv -b -m copy -a "src=$temp_dir/$bootstrap_ip/tmp/ceph.client.admin.keyring dest=/etc/ceph/ceph.client.admin.keyring group=root owner=root mode=600" ceph_nodes

In [ ]:
!ansible -CDv -b -m copy -a "src=$temp_dir/$bootstrap_ip/tmp/ceph.conf dest=/etc/ceph/ceph.conf group=root owner=root mode=644" ceph_nodes

In [ ]:
!ansible -b -m copy -a "src=$temp_dir/$bootstrap_ip/tmp/ceph.client.admin.keyring dest=/etc/ceph/ceph.client.admin.keyring group=root owner=root mode=600" ceph_nodes

In [ ]:
!ansible -b -m copy -a "src=$temp_dir/$bootstrap_ip/tmp/ceph.conf dest=/etc/ceph/ceph.conf group=root owner=root mode=644" ceph_nodes

In [ ]:
!ansible -b -m shell -a 'ls -la /etc/ceph' ceph_nodes

## ovs_ctrl再起動

In [ ]:
!ansible -b -m service -a "name=ovs_ctrl enabled=yes state=restarted" ceph_nodes

In [ ]:
!ansible -b -m shell -a "systemctl status ovs_ctrl" ceph_nodes

In [ ]:
!ansible -b -m service -a "name=ovs_ctrl enabled=yes state=restarted" bootstrap

In [ ]:
!ansible -b -m shell -a "systemctl status ovs_ctrl" bootstrap

# 後始末

一時ディレクトリを削除する。

In [ ]:
!rm -fr $temp_dir